Welcome to the tutoriol of Birdwatcher! 

In this first notebook, we'll introduce how you can load a video as VideoFileStream object. We also show how to look at the video frames, explain what a Frames object is, and how to apply some basic manipulations to the video frames.

In [ ]:
import birdwatcher as bw
from birdwatcher.plotting import imshow_frame # birdwatcher has vizualization tools
%matplotlib inline

### Create a video object

A short videofile is distributed with Birdwatcher which you can use to run the notebooks. But you can also enter a pathname to your own videofile.

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zf20s_low.mp4')
vfs # this is a VideoFileStream

In [ ]:
vfs.streammetadata

Some useful properties and methods:

In [ ]:
print(f"average frame rate: {vfs.avgframerate}")
print(f"number of frames (reported): {vfs.nframes}")
print(f"counted number of frames: {vfs.count_frames()}")
print(f"duration: {vfs.duration}")
print(f"framewidth: {vfs.framewidth}")
print(f"frameheight: {vfs.frameheight}")
print(f"framesize: {vfs.framesize}")

### Look at video frames

Look at video in separate window:

In [ ]:
vfs.show()   # press 'q' if you want to quit video before end

Get a frame at a given time, say at 10 s:

In [ ]:
frame = vfs.get_frameat('00:10.')
frame # numpy array, uint8

Get a frame by number. This is inefficient for long videos, but precise. The video is decoded up to that point.

In [ ]:
frame = vfs.get_frame(250)

In [ ]:
imshow_frame(frame)

### Iterate Frames in video file

In [ ]:
# only iterate first 20 frames
for frame in vfs.iter_frames(nframes=20):
    print(frame.shape, end = ', ')

For most processing it is important to realize that the `iter_frames` method returns a Frames iterator object. This is a central type in Birdwatcher as it has a lot of useful methods for processing, analysis, and writing videos. Many methods of a Frames object return another Frames object. This way you can quickly setup a processing pipeline before doing something final such as writing a video or analysis.

In [ ]:
frames = vfs.iter_frames()
frames

In [ ]:
frames = frames.blur((10,10)).togray().draw_framenumbers()
frames

Get information, including the methods applied to a Frames object:

In [ ]:
frames.get_info()

Get a sneak preview of the manipulations:

In [ ]:
frame = frames.peek_frame()
imshow_frame(frame)

Save as a video file:

In [ ]:
vfs_processed = frames.tovideo('output/processed_frames.mp4', framerate=vfs.avgframerate)

In [ ]:
vfs_processed.show() # press 'q' if you want to quit video before end

Note that, because a Frames object is an iterator, the object will be exhausted after iterating through a Frames object, such as when writing to a video or calculating the mean frame (see below). If you try to apply another method on an 'empty' Frames iterator, this will raise a `StopIteration`. To set-up a new analysis, you will need to create the Frames iterator again, e.g. by running the `iter_frames` method again.

In [ ]:
frame = vfs.iter_frames().calc_meanframe() # calculate the mean frame
imshow_frame(frame)

### Select video fragment

To select a video fragment specify the start time and the number of frames when using `iter_frames`:

In [ ]:
startat = '00:00:02'   # in HOURS:MM:SS
nframes = 100

frames = vfs.iter_frames(startat=startat, nframes=nframes)

Using the number of frames to specificy the duration of the video fragment is precise. But, if you want to use the duration in time to select a video fragment, just approximate the number of frames:

In [ ]:
duration = 15   # in seconds
nframes = int(vfs.avgframerate*duration)
print(f'A video fragment of {duration} seconds with framerate {vfs.avgframerate} per second corresponds with {nframes} frames.')

Look at the chosen video fragment:

In [ ]:
frames.show(framerate=150)

### Select region of interest

If there is a region in the video that you don't want to track, e.g. where the subject cannot move to, or a region with movement (such as a timer), you can exclude this part of each frame.

To specify the coordinates of the rectangle that should be excluded or included, it's useful to know the framewidth and height of the video:

In [ ]:
print(f"Vertical coordinates h1, h2 = 0, {vfs.frameheight}")
print(f"Horizontal coordinates w1, w2 = 0, {vfs.framewidth}")

Specify region of interest, so that only in this rectangular region movement detection is done. Using`imshow_frame` you can also see the coordinates along the horizontal and vertical axes.

In [ ]:
# specify h1, h2, w1, w2
roi = (10, 570, 10, 1250)   # or choose None to use the whole frame

# show roi in frame
frame = vfs.iter_frames().peek_frame()
imshow_frame(frame, draw_rectangle=roi)

Or, define a region that should be excluded:

In [ ]:
# specify h1, h2, w1, w2
nroi = (600, 720, 0, 1280)   # or choose None to use the whole frame

# show nroi in frame
frame = vfs.iter_frames().peek_frame()
imshow_frame(frame, draw_rectangle=nroi)